In [27]:
from lifelines.datasets import load_rossi
from lifelines import CoxPHFitter
from scipy.stats import norm

In [2]:
%matplotlib inline
import os
from os import path
from glob import glob
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from utils import *

In [25]:
def invers_norm(data):
    rank = np.argsort(data)
    q = [0 for _ in range(len(rank))]
    for i,k in enumerate(rank):
        q[k] = i+1
    q = (np.array(q)-0.5) /len(q)
    return norm.ppf(q)

In [14]:
# cancer_type = 'Lung'
# cluster_num = 3
# rand_run = 4
# clinical_filename = '/home/tunthey/Dropbox/Tunde/analysis/new_datasets/' + cancer_type + '/' + cancer_type +  '_clinical_data.txt'
# drug_filename = '/home/tunthey/Dropbox/Tunde/analysis/new_datasets/' + cancer_type + '/' + cancer_type +  '_drug_data.txt'
# cluster_label = '/home/tunthey/Dropbox/Tunde/analysis/new_results/MVKKM_newdata/labels/' + cancer_type + '_mvkkm_labels_K' +  str(cluster_num) + '_random_init_run' + str(rand_run) + '_rbf_kernel_old_gamma_with_shankar_distance_second_derv.txt'
# pam50_file = '/home/tunthey/Dropbox/Tunde/analysis/PAM50/bioclassifier_R/Breastcancerpan50subtyperesult_pam50scores.txt'


In [34]:
# cluster_l = open(cluster_label).readlines()
# cluster_list = [s.rstrip() for s in cluster_l]
# np.array(cluster_list)
# cluster_list

In [4]:
# rossi_dataset = load_rossi()
# cph = CoxPHFitter()
# cph.fit(rossi_dataset, duration_col='week', event_col='arrest')

# cph.plot()


In [3]:
with open('../data/exp_final_gene_list.txt', 'r') as f:
    final_genes= [l.rstrip() for l in f.readlines()]
with open('../data/hint_inters_pan.txt', 'r') as f:
    hint_u_pan= [l.rstrip() for l in f.readlines()]

In [4]:
cancer_types = ['BLCA','BRCA','CRC', 'GBM', 'HNSC', 'KIRC', 'LAML', 'LUAD', 'LUSC', 'OV', 'UCEC']

In [5]:
comps = []
significant_cancers =[]
with open('../hint/out/cancer_subtype_test_results/mutex_t07_nsep_cov_nsep/cc_n100_cancer_subtype_tests.txt', 'r') as f:
    lines = f.readlines()[:]
    for i in range(0,len(lines), 12):
        comp = lines[i].strip().split('[')[-1].split(',')
        comp = [c.replace(']', '').replace('\'', '').strip() for c in comp]
        comps.append(comp)
        type_interest = [j-1 for j in range(1,12) if len(lines[i+j].split()) ==8]
        significant_cancers.append(type_interest)
        

In [6]:
cancer2modules = {}
for i,m in enumerate(comps):
    for t in significant_cancers[i]:
        if cancer_types[t] in cancer2modules.keys():
            cancer2modules[cancer_types[t]].append(m)
        else:
            cancer2modules[cancer_types[t]] = [m]

In [7]:
module2id = {str(m):i for i,m in enumerate(comps)}
id2module = {i:str(m) for i,m in enumerate(comps)}

In [8]:
cancer2modules.keys()

dict_keys(['HNSC', 'LAML', 'LUAD', 'KIRC', 'OV', 'BRCA', 'GBM', 'LUSC', 'UCEC', 'CRC'])

In [9]:
id2module[0]

"['POLR2A', 'SMARCA2', 'SMARCA4']"

In [10]:
out_path = '../hint/out/'
data_path = '../data/'
data_files = 'pancancer/'
clinic_path = data_path+'clinical_data/'
clinic_file = '{}_clinical_data.txt'
files = glob(data_path+data_files+'*/[!MANIFEST]*.txt')

In [11]:
cancer2file = {c.split('/')[-1].split('.')[0]:c for c in files}

In [12]:
df = pd.read_csv(cancer2file['OV'], delimiter = '\t', low_memory=False)
df.drop(df.index[0], inplace=True)
gene_and_id = df.iloc[:, 0].values
df.index = final_genes
df.drop(df.columns[0], axis=1, inplace= True)
df = df.loc[list(hint_u_pan)]

In [13]:
df_ = df.T
df_.index.names = ['patients']
df_.columns.name = 'genes'

In [15]:
ids= [s.split('-')[3] for s in df_.index]

In [16]:
np.unique(ids, return_counts=True)

(array(['01A', '01B', '01C', '01R', '02A'], dtype='<U3'),
 array([292,   9,   1,   1,   4]))

In [17]:
names = ['-'.join(x.split('-')[:3]) for x in df_.index]
unique = np.unique(names, return_counts=True)

In [18]:
keep_patient = [x.split('-')[3][:2] == '01' for x in df_.index]
df_ = df_.loc[keep_patient]

In [19]:
df_.shape

(303, 9523)

In [20]:
df_.index = ['-'.join(x.split('-')[:3]) for x in df_.index]
df_.index.names = ['patients']

In [22]:
# df_.loc['TCGA-04-1348']

In [23]:
# np.unique(df_.index.values, return_counts=True)

In [330]:
from rank_based_inverse_normal_transformation import *

In [28]:
df3 = df_.apply(invers_norm, axis = 0)

In [46]:
df2 = df_clinical.join(df3)

In [47]:
df2 = df2.loc[~df2[df2.columns[4:]].isnull().any(axis=1)]

In [434]:
df_.head()

genes,NUP98,ZKSCAN5,LRRC7,RHO,WASH3P,KIF21B,LASP1,C2orf62,FSTL3,ITPR3,...,LCA5,CACNB2,AMPH,GNG10,COMTD1,KNG1,RRAGB,RRAGC,TMED2,TRPT1
patients,,,,,,,,,,,,,,,,,,,,,
TCGA-04-1348-01A-01R-1565-13,2595.3213,412.4765,3.4956,0.2689,739.4461,93.3047,4250.3361,8.0667,213.7671,2736.2087,...,33.8801,13.4445,6.9911,764.6276,434.5254,0.0000,267.0073,219.0777,9459.5321,670.6104
TCGA-04-1357-01A-01R-1565-13,1252.6042,501.9531,0.0000,0.0000,393.2292,315.7552,4565.7552,3.9062,289.7135,3390.9115,...,73.5677,20.1823,234.3750,542.3177,326.8229,0.0000,261.0677,281.2500,6087.8906,571.6146
TCGA-04-1362-01A-01R-1565-13,1877.5287,223.7281,1.9858,4.6334,715.5327,176.4010,5779.8628,2.6477,183.6821,4397.7263,...,228.6925,17.2099,53.6153,1664.7223,439.8441,0.3310,285.2864,311.7135,8183.6159,894.9123
TCGA-04-1364-01A-01R-1565-13,2235.0277,694.5505,0.0000,0.0000,1021.4725,360.4200,3575.3659,26.2895,289.1840,818.3611,...,172.1535,17.3850,17.3850,652.1481,981.1903,0.0000,80.9885,162.6978,6663.1049,552.9266
TCGA-04-1365-01A-01R-1565-13,1647.1297,423.0827,62.6789,2.1094,813.9220,86.4849,5431.9723,8.1362,231.1285,3265.2192,...,106.0720,12.9577,29.8328,809.4018,467.6812,0.0000,155.7933,278.1377,9837.5772,1042.0371


In [49]:
df2.shape

(301, 9527)

In [206]:
for module in cancer2modules['LAML']:

        module_id = module2id[str(module)]
#         if not os.path.exists(results_path+'{}/'.format(module_id)):
#             os.mkdir(results_path+'{}/'.format(module_id))
#         csv_path = results_path+'{}/{}.csv'.format(module_id, cancer)
        df2 = df2.astype('float')
        tmp_df  = df2[module+['event','time_to_event']]
        cph = CoxPHFitter()
        cph.fit(tmp_df, duration_col='time_to_event', event_col='event', show_progress=False)
        s = cph.summary

In [40]:
clinical_data = clinic_file.format('OV')
df_clinical = pd.read_csv(clinic_path+clinical_data,usecols = ['patient_id','age', 'gender','event','time_to_event'], delimiter = '\t', low_memory=False)

In [41]:
df_clinical = df_clinical.set_index('patient_id')
df_clinical.index.names = ['patients']
#df_clinic.set_index('patient_id')

In [42]:
df_clinical =df_clinical[~df_clinical.isnull().any(axis=1)]

In [43]:
df_clinical[df_clinical.isnull().any(axis=1) == True]

,event,time_to_event,age,gender
patients,,,,


In [44]:
df_clinical.shape

(575, 4)

In [45]:
df_clinical.head()

,event,time_to_event,age,gender
patients,,,,
TCGA-3P-A9WA,1,243.0,55,1
TCGA-59-A5PD,0,624.0,55,1
TCGA-5X-AA5U,1,277.0,61,1
TCGA-04-1331,0,1336.0,78,1
TCGA-04-1332,0,1247.0,70,1


In [40]:
df1 = df_.astype('float')

In [53]:
tmp_df.isnull().any()

NOTCH1           False
age              False
gender           False
event            False
time_to_event    False
dtype: bool

In [54]:
tmp_df  = df2[['NOTCH1']+['age','event','time_to_event']].astype('float')
cph = CoxPHFitter()
cph.fit(tmp_df, duration_col='time_to_event', event_col='event', show_progress=True)

Iteration 1: norm_delta = 0.25530, step_size = 0.95000, ll = -632.32135, newton_decrement = 3.60896, seconds_since_start = 0.0
Iteration 2: norm_delta = 0.01077, step_size = 0.95000, ll = -628.74333, newton_decrement = 0.00662, seconds_since_start = 0.0
Iteration 3: norm_delta = 0.00055, step_size = 0.95000, ll = -628.73672, newton_decrement = 0.00002, seconds_since_start = 0.1
Iteration 4: norm_delta = 0.00003, step_size = 0.95000, ll = -628.73671, newton_decrement = 0.00000, seconds_since_start = 0.1
Convergence completed after 4 iterations.


<lifelines.CoxPHFitter: fitted with 301 observations, 171 censored>

In [377]:
s = cph.summary

In [378]:
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 152 observations, 32 censored>
      duration col = 'time_to_event'
         event col = 'event'
number of subjects = 152
  number of events = 120
    log-likelihood = -471.96
  time fit was run = 2019-01-24 06:22:22 UTC

---
        coef  exp(coef)  se(coef)     z      p  log(p)  lower 0.95  upper 0.95    
NOTCH1 -0.10       0.90      0.10 -1.06   0.29   -1.24       -0.29        0.09    
age     0.03       1.03      0.01  3.66 <0.005   -8.29        0.01        0.04  **
gender  0.12       1.13      0.19  0.62   0.53   -0.63       -0.26        0.50    
---
Signif. codes: 0 '***' 0.0001 '**' 0.001 '*' 0.01 '.' 0.05 ' ' 1

Concordance = 0.61
Likelihood ratio test = 15.49 on 3 df, log(p)=-6.54


In [379]:
s

,coef,exp(coef),se(coef),z,p,log(p),lower 0.95,upper 0.95
NOTCH1,-0.101338,0.903628,0.095587,-1.060160,0.289072,-1.241081,-0.288685,0.086010
age,0.027775,1.028164,0.007584,3.662381,0.000250,-8.294521,0.012911,0.042639
gender,0.120577,1.128147,0.194207,0.620868,0.534687,-0.626075,-0.260061,0.501214


In [279]:
from cox_utils import *

In [282]:
# Data is [[Patient ID, time (days), vital status, 0, sex, age at diagnosis],...]
clinical = np.array(load_clinical())

In [284]:
clinical[:5]

array([['TCGA-02-0001', '358', 'Dead', '0', '1', '44'],
       ['TCGA-02-0003', '144', 'Dead', '0', '0', '50'],
       ['TCGA-02-0004', '345', 'Dead', '0', '0', '59'],
       ['TCGA-02-0006', '558', 'Dead', '0', '1', '56'],
       ['TCGA-02-0007', '705', 'Dead', '0', '1', '40']], dtype='<U12')

In [292]:
data = {'patients':clinical[:,0],
       'time_to_event':clinical[:,1],
        'event': [1 if s == 'Dead' else 0 for s in clinical[:,2]],
        'gender': clinical[:,4],
        'age': clinical[:,5]
        
       }

In [298]:
df_clinical = pd.DataFrame(data,)

In [303]:
df_clinical = df_clinical.set_index('patients')
df_clinical =df_clinical[~df_clinical.isnull().any(axis=1)]

In [301]:
df_clinical.shape

(595, 5)

In [410]:
df_clinical.head()

,time_to_event,event,gender,age
patients,,,,
TCGA-02-0001,358,1,1,44
TCGA-02-0003,144,1,0,50
TCGA-02-0004,345,1,0,59
TCGA-02-0006,558,1,1,56
TCGA-02-0007,705,1,1,40


In [317]:
df_clinical.loc['TCGA-14-1829']

time_to_event    218
event              0
gender             0
age               57
Name: TCGA-14-1829, dtype: object

In [328]:
df_.loc['TCGA-14-1829']['NOTCH1']

'378.6248'

In [428]:
clinical_names = set(df_clinical.index.values)
exp_names = set(df_.index)

In [412]:
len(clinical_names)

595

In [430]:
clinical_names.intersection(exp_names)

set()

In [345]:
st = '0.39460475 0.15091132 0.43753911 0.65392753 0.51048247 0.47433630 0.00711546 0.12695813 0.74702367 0.9163484'
data = np.array([float(s) for s in st.split()])

In [349]:
rank = np.argsort(data)
rank

array([6, 7, 1, 0, 2, 5, 4, 3, 8, 9])

In [357]:
q = [0 for _ in range(len(rank))]
for i,k in enumerate(rank):
    q[k] = i+1
q = (np.array(q)-0.5) /len(q)
q

array([0.35, 0.25, 0.45, 0.75, 0.65, 0.55, 0.05, 0.15, 0.85, 0.95])

In [354]:
from scipy.stats import norm

In [358]:
norm.ppf(q)

array([-0.38532047, -0.67448975, -0.12566135,  0.67448975,  0.38532047,
        0.12566135, -1.64485363, -1.03643339,  1.03643339,  1.64485363])

In [359]:
def invers_norm(data):
    rank = np.argsort(data)
    q = [0 for _ in range(len(rank))]
    for i,k in enumerate(rank):
        q[k] = i+1
    q = (np.array(q)-0.5) /len(q)
    return norm.ppf(q)